In [ ]:
import numpy as np
import cv2
import os

 
images_folder_with_fix_tail = "N2-adults-50um-SRx5-fix-tail-crop-images-correct-angle-folders"
images_folder = './N2-adults-50um-SRx5-crop-images-correct-angle-folders'

seg_folder_with_fix_tail = "N2-adults-50um-SRx5-fix-tail-crop-seg-correct-angle-folders"
seg_folder = './N2-adults-50um-SRx5-crop-seg-correct-angle-folders'

sub_folders = os.listdir(images_folder)

print(sub_folders)

pointcloud_folder = "N2-adults-50um-SRx5-3D-pointcloud"


try:
    os.makedirs(pointcloud_folder)
    print(f"Folder '{pointcloud_folder}' created successfully.")
except FileExistsError:
    print(f"Folder '{pointcloud_folder}' already exists.")   


for sub_folder in sub_folders:
    sub_pointcloud_folder = pointcloud_folder + '/' + sub_folder
    try:
        os.makedirs(sub_pointcloud_folder)
    except FileExistsError:
        pass


In [ ]:
def compute_np_iu(seg, gt):
    intersection = (seg & gt).astype(np.float32).sum()
    union = (seg | gt).astype(np.float32).sum()
    return intersection, union

def compute_np_iou(seg, gt):
    intersection, union = compute_np_iu(seg, gt)
    iou = (intersection + 1e-6) / (union + 1e-6)
    return iou

In [ ]:
import math
import open3d
from copy import deepcopy


for sub_folder in sub_folders:

    
    
    print(sub_folder)

    images_with_fix_tail_list_dir = images_folder_with_fix_tail+'/'+sub_folder
    seg_with_fix_tail_images_list_dir = seg_folder_with_fix_tail+'/'+sub_folder


    images_list_dir = images_folder+'/'+sub_folder
    seg_images_list_dir = seg_folder+'/'+sub_folder

    images_list = os.listdir(images_list_dir)
    seg_images_list = os.listdir(seg_images_list_dir)




    
    print('，')
    best_match_index = 0
    best_match_score = 0
    for image_name in images_list:

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))
            
            if image_index==0:

                img_tempate = cv2.imread(images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE)

            elif image_index>200:
                img = cv2.imread(images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE)

                res = cv2.matchTemplate(img, img_tempate, cv2.TM_CCOEFF_NORMED)

                score = res[0][0]
                if score > best_match_score:
                    best_match_index = image_index
                    best_match_score = score
                    

    print('best_match_index = ',best_match_index, ', best_match_score = ' ,best_match_score)


    
    print(', offset')
    cloud_points = None
    degree_per_frame = 360.0/(best_match_index+1)

    for image_name in images_list:

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))
            
            
            if image_index >= 0 and image_index <= best_match_index:

                mask = cv2.imread(seg_with_fix_tail_images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE)

                h,w = mask.shape

                

                
                contours = cv2.findContours(
                    mask.astype("uint8"), cv2.RETR_CCOMP,
                    cv2.CHAIN_APPROX_NONE)[-2]


                
                c_index = 0
                for i in range(len(contours)):

                    mu=cv2.moments(contours[i],False)
                    center_x = int(mu['m10'] / (mu['m00']+0.00000001))
                    center_y = int(mu['m01'] / (mu['m00']+0.00000001))
                    area = cv2.contourArea(contours[i])
                    if area > 100000:
                        c_index = i

                


                
                

                

                
                contours_by_height = {}
                
                for i in range(h):
                    contours_by_height[i] = []

                    condition = (contours[c_index][:, :, 1] == i)  
                    filtered_data = contours[c_index][condition]  

                    contours_by_height[i] = filtered_data

                

                xyz_point = []
                for data_y in contours_by_height:

                    
                    length = len(contours_by_height[data_y])

                    
                    if length > 0:
                        if length == 1:
                            width = 1
                        else:

                            
                            x_coords = contours_by_height[data_y][:,0]
                            
                            max_x = np.max(x_coords)
                            min_x = np.min(x_coords)
                            width = max_x - min_x

                        r = width/2.0 

                        theta = math.radians(degree_per_frame * image_index)
                        
                        x = -r * math.cos(theta)
                        y = data_y
                        z = r * math.sin(theta)
                        xyz_point.append([x,y,z])

                xyz_point = np.array(xyz_point)

                pcd = open3d.geometry.PointCloud()
                pcd.points = open3d.utility.Vector3dVector(xyz_point)


                if cloud_points == None:
                    
                    pcd.paint_uniform_color([1,0,0])
                    cloud_points = pcd
                else:
                    
                    if image_index == int(best_match_index/4):
                        pcd.paint_uniform_color([0,1,0])
                    else:
                        pcd.paint_uniform_color([1,1,1])

                    cloud_points += pcd


    
    open3d.io.write_point_cloud(pointcloud_folder+'/'+sub_folder+'/before_offset.pcd', cloud_points , write_ascii=True)  

    
    print(', , offset')

    sections_num = 5 
    sections_max_iou = [0 for i in range(sections_num)]
    sections_points = [None for i in range(sections_num)]
    interval_90_degree_imgs_num = int(best_match_index/4)
    max_mean_iou = 0 

    
    for image_name in images_list:

        pcd_offset = deepcopy(cloud_points)

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))

            if image_index % int(best_match_index/36) != 0: 
                continue
            
            
            
            

            
            if image_index < interval_90_degree_imgs_num*3: 

                image_name_list_90deg = [image_name,str(image_index+interval_90_degree_imgs_num).zfill(3)+'.png']
                print('image_name_list_90deg = ',image_name_list_90deg)

                for img_90deg in image_name_list_90deg:

                    image_index = int(img_90deg.replace('.png','')) 
                    img  = cv2.imread(images_with_fix_tail_list_dir + '/' + img_90deg, cv2.IMREAD_GRAYSCALE) 
                    mask = cv2.imread(seg_with_fix_tail_images_list_dir + '/' + img_90deg, cv2.IMREAD_GRAYSCALE) 

                    h,w = mask.shape

                    
                    contours = cv2.findContours(
                        mask.astype("uint8"), cv2.RETR_CCOMP,
                        cv2.CHAIN_APPROX_NONE)[-2]

                    
                    c_index = 0
                    for i in range(len(contours)):

                        mu=cv2.moments(contours[i],False)
                        center_x = int(mu['m10'] / (mu['m00']+0.00000001))
                        center_y = int(mu['m01'] / (mu['m00']+0.00000001))
                        area = cv2.contourArea(contours[i])
                        if area > 100000:
                            c_index = i

                    
                    contours_by_height = {}
                    
                    for i in range(h):
                        contours_by_height[i] = []
                        condition = (contours[c_index][:, :, 1] == i)  
                        filtered_data = contours[c_index][condition]  
                        contours_by_height[i] = filtered_data


                    
                    R = pcd_offset.get_rotation_matrix_from_xyz((0, -math.radians(degree_per_frame * image_index), 0))
                    pcd_offset.rotate(R)

                    pcd_offset_points = np.asarray(pcd_offset.points)

                    for data_y in contours_by_height:
                        
                        
                        length = len(contours_by_height[data_y])

                        
                        if length > 0:
                            if length == 1:
                                width = 1
                                
                                x_coords = contours_by_height[data_y][:,0]
                                
                                max_x = np.max(x_coords)
                                min_x = np.min(x_coords)
                                width = 1/2
                            else:
                                
                                x_coords = contours_by_height[data_y][:,0]
                                
                                max_x = np.max(x_coords)
                                min_x = np.min(x_coords)
                                width = max_x - min_x

                            
                            r = width/2.0 
                            delta_x = min_x + r -239 

                            
                            condition = (pcd_offset_points[:,1] == data_y)  

                            
                            

                            
                            pcd_offset_points[condition, 0] += delta_x
                            

                    
                    pcd_offset = open3d.geometry.PointCloud()
                    
                    pcd_offset.points = open3d.utility.Vector3dVector(pcd_offset_points)
                    
                    R = pcd_offset.get_rotation_matrix_from_xyz((0, math.radians(degree_per_frame * image_index), 0))
                    pcd_offset.rotate(R)

                open3d.io.write_point_cloud(pointcloud_folder+'/'+sub_folder+'/after_offset.pcd', pcd_offset , write_ascii=True)  


                
                pcd_offset_file = open3d.io.read_point_cloud(pointcloud_folder+'/'+sub_folder+'/after_offset.pcd')
                mean_iou = []
                sections_mean_iou = [[] for i in range(sections_num)]

                for image_name in images_list:

                    if image_name.endswith('.png'):
                        image_index = int(image_name.replace('.png',''))


                        
                        if image_index >= 0 and image_index <= best_match_index:

                            if image_index % int(best_match_index/12) == 0 and image_index <= (best_match_index/2):
                                pass
                            else:
                                continue

                            

                            
                            img  = cv2.imread(images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE) 
                            mask = cv2.imread(seg_images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE) 

                            
                            h,w = mask.shape
                            contours = cv2.findContours(
                                mask.astype("uint8"), cv2.RETR_CCOMP,
                                cv2.CHAIN_APPROX_NONE)[-2]

                            c_index = 0
                            for i in range(len(contours)):

                                mu=cv2.moments(contours[i],False)
                                area = cv2.contourArea(contours[i])
                                if area > 100000:
                                    c_index = i

                            mu=cv2.moments(contours[c_index],False)
                            mask_center_x = int(mu['m10'] / (mu['m00']+0.00000001))
                            mask_center_y = int(mu['m01'] / (mu['m00']+0.00000001))
                            mask_x, mask_y, mask_w, mask_h = cv2.boundingRect(contours[c_index])
                            mask_bottom_y = mask_y + mask_h


                            
                            pcd_offset = deepcopy(pcd_offset_file)
                            R = pcd_offset.get_rotation_matrix_from_xyz((0, -math.radians(degree_per_frame * image_index), 0))
                            pcd_offset.rotate(R)
                            pcd_offset_points = np.asarray(pcd_offset.points).astype(np.int32)
                            pcd_offset_projection_points = pcd_offset_points[:,0:2]
                            pcd_offset_projection_points[:,0] += 249
                            projection_image = np.zeros([2600,480])
                            pcd_offset_projection_points = np.unique(pcd_offset_projection_points, axis=0)

                            
                            valid_indices = np.logical_and.reduce((
                                pcd_offset_projection_points[:, 0] >= 0,
                                pcd_offset_projection_points[:, 0] < 480,
                                pcd_offset_projection_points[:, 1] >= 0,
                                pcd_offset_projection_points[:, 1] < 2600
                            ))

                            pcd_offset_projection_points = pcd_offset_projection_points[valid_indices]

                            projection_image[pcd_offset_projection_points[:, 1], pcd_offset_projection_points[:, 0]] = 255

                            
                            contours_prj = cv2.findContours(
                                projection_image.astype("uint8"), cv2.RETR_CCOMP,
                                cv2.CHAIN_APPROX_NONE)[-2]

                            prj_c_index = 0
                            for i in range(len(contours_prj)):

                                mu=cv2.moments(contours_prj[i],False)
                                area = cv2.contourArea(contours_prj[i])
                                if area > 50000:
                                    prj_c_index = i

                            
                            projection_image = np.zeros_like(projection_image)
                            cv2.fillPoly(projection_image, [contours_prj[prj_c_index]], (255))   


                            mu=cv2.moments(contours_prj[prj_c_index],False)
                            projection_image_center_x = int(mu['m10'] / (mu['m00']+0.00000001))
                            projection_image_center_y = int(mu['m01'] / (mu['m00']+0.00000001))
                            projection_image_x, projection_image_y, projection_image_w, projection_image_h = cv2.boundingRect(contours[c_index])
                            projection_image_bottom_y = projection_image_y + projection_image_h



                            
                            tx = mask_center_x-projection_image_center_x 
                            
                            ty = mask_bottom_y-projection_image_bottom_y 

                            affine_arr = np.float32([[1,0,tx],[0,1,ty]])
                            projection_image = cv2.warpAffine(projection_image,affine_arr,(projection_image.shape[1],projection_image.shape[0]))  

                            
                            
                            

                            
                            projection_image[0:mask_y,:] = 0
                            mask[0:mask_y,:] = 0

                            
                            iou = compute_np_iou(projection_image.astype("uint8"), mask.astype("uint8"))
                            
                            mean_iou.append(iou)

                
                if np.mean(np.array(mean_iou)) > max_mean_iou:
                    max_mean_iou = np.mean(np.array(mean_iou))
                    open3d.io.write_point_cloud(pointcloud_folder+'/'+sub_folder+'/'+'best_iou_after_offset.pcd', pcd_offset_file , write_ascii=True)  

                    
                    for image_name in images_list:

                        if image_name.endswith('.png'):
                            image_index = int(image_name.replace('.png',''))


                            
                            if image_index >= 0 and image_index <= best_match_index:

                                if image_index % int(best_match_index/12) == 0 and image_index <= (best_match_index/2):
                                    pass
                                else:
                                    continue

                                

                                
                                img  = cv2.imread(images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE) 
                                mask = cv2.imread(seg_images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE) 

                                
                                h,w = mask.shape
                                contours = cv2.findContours(
                                    mask.astype("uint8"), cv2.RETR_CCOMP,
                                    cv2.CHAIN_APPROX_NONE)[-2]

                                c_index = 0
                                for i in range(len(contours)):

                                    mu=cv2.moments(contours[i],False)
                                    area = cv2.contourArea(contours[i])
                                    if area > 100000:
                                        c_index = i

                                mu=cv2.moments(contours[c_index],False)
                                mask_center_x = int(mu['m10'] / (mu['m00']+0.00000001))
                                mask_center_y = int(mu['m01'] / (mu['m00']+0.00000001))
                                mask_x, mask_y, mask_w, mask_h = cv2.boundingRect(contours[c_index])
                                mask_bottom_y = mask_y + mask_h


                                
                                pcd_offset = deepcopy(pcd_offset_file)
                                
                                cl,index = pcd_offset.remove_statistical_outlier(nb_neighbors = 50,std_ratio= 1.0)
                                pcd_offset = pcd_offset.select_by_index(index)


                                R = pcd_offset.get_rotation_matrix_from_xyz((0, -math.radians(degree_per_frame * image_index), 0))
                                pcd_offset.rotate(R)
                                pcd_offset_points = np.asarray(pcd_offset.points).astype(np.int32)
                                pcd_offset_projection_points = pcd_offset_points[:,0:2]
                                pcd_offset_projection_points[:,0] += 249
                                projection_image = np.zeros([2600,480])
                                pcd_offset_projection_points = np.unique(pcd_offset_projection_points, axis=0)

                                
                                valid_indices = np.logical_and.reduce((
                                    pcd_offset_projection_points[:, 0] >= 0,
                                    pcd_offset_projection_points[:, 0] < 480,
                                    pcd_offset_projection_points[:, 1] >= 0,
                                    pcd_offset_projection_points[:, 1] < 2600
                                ))

                                pcd_offset_projection_points = pcd_offset_projection_points[valid_indices]

                                projection_image[pcd_offset_projection_points[:, 1], pcd_offset_projection_points[:, 0]] = 255

                                
                                contours_prj = cv2.findContours(
                                    projection_image.astype("uint8"), cv2.RETR_CCOMP,
                                    cv2.CHAIN_APPROX_NONE)[-2]

                                prj_c_index = 0
                                for i in range(len(contours_prj)):

                                    mu=cv2.moments(contours_prj[i],False)
                                    area = cv2.contourArea(contours_prj[i])
                                    if area > 50000:
                                        prj_c_index = i

                                
                                projection_image = np.zeros_like(projection_image)
                                cv2.fillPoly(projection_image, [contours_prj[prj_c_index]], (255))   


                                mu=cv2.moments(contours_prj[prj_c_index],False)
                                projection_image_center_x = int(mu['m10'] / (mu['m00']+0.00000001))
                                projection_image_center_y = int(mu['m01'] / (mu['m00']+0.00000001))
                                projection_image_x, projection_image_y, projection_image_w, projection_image_h = cv2.boundingRect(contours[c_index])
                                projection_image_bottom_y = projection_image_y + projection_image_h


                                
                                tx = mask_center_x-projection_image_center_x 
                                
                                ty = mask_bottom_y-projection_image_bottom_y 

                                affine_arr = np.float32([[1,0,tx],[0,1,ty]])
                                projection_image = cv2.warpAffine(projection_image,affine_arr,(projection_image.shape[1],projection_image.shape[0]))  


                                cv2.imwrite(pointcloud_folder+'/'+sub_folder+'/'+str(image_index)+'_projection_image'+'.png',projection_image)
                                cv2.imwrite(pointcloud_folder+'/'+sub_folder+'/'+str(image_index)+'_mask_image'+'.png',mask)
                                cv2.imwrite(pointcloud_folder+'/'+sub_folder+'/'+str(image_index)+'_origin_image'+'.png',img)


                print('mean_iou, max_mean_iou = ', np.mean(np.array(mean_iou)), max_mean_iou)

